In [1]:
from torch.nn import functional as F
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import models
from torchvision import datasets
from torchvision import transforms as T

import numpy as np
import torchutils as tu
import matplotlib.pyplot as plt

import mlflow
from mlflow.models import infer_signature
from PIL import Image

In [2]:
use_mlflow = True
mlflow_experiment_name = 'INTEL-CNN-Experiment'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
config = {
    'device': DEVICE,
    'batch_size': 1024,
    'learning_rate': 0.001,
    'epochs': 5,
    'data_dir': '../data/',
    'log_dir': './runs/eperiment2',
    'transformations': [
        'resize', 'normalize'
    ]
}

DEVICE

'cuda'

In [3]:
train_transforms = T.Compose(
    [
        T.Resize(256),                
        T.RandomResizedCrop(224, scale=(0.8, 1.0)),  
        T.RandomRotation(15),         
        T.RandomHorizontalFlip(p=0.5),
        T.ColorJitter(
            brightness=0.1,    
            contrast=0.1,      
        ),
        T.ToTensor(),
        T.Normalize(
            (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
        )
    ]
)
valid_transforms = T.Compose(
    [
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(
            (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
        )
    ]
)

In [4]:
train_dataset = datasets.ImageFolder(
    config['data_dir']+ 'seg_train/seg_train',
    transform=train_transforms
    )

valid_dataset = datasets.ImageFolder(
    config['data_dir']+ 'seg_test/seg_test',
    transform=train_transforms
    )

In [5]:
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False)

In [6]:
labels = {
    0: 'buildings',
    1: 'forest',
    2: 'glacier',
    3: 'mountain',
    4: 'sea',
    5: 'street'
}

decode = lambda x: labels[x]
decode(4)

'sea'

In [7]:
model = models.resnet50(weights='DEFAULT')

In [8]:
for param in model.parameters():
    param.requires_grad = False
       
model.fc = nn.Linear(2048, 6)
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
optimizer = torch.optim.Adam(model.parameters(), config['learning_rate'])
criterion = torch.nn.CrossEntropyLoss()

In [10]:
tu.get_model_summary(model, torch.randn(config['batch_size'], 3, 224, 224, device=DEVICE))

Layer                                         Kernel                Output           Params               FLOPs
0_conv1                                      [3, 64, 7, 7]   [1024, 64, 112, 112]       9,408   120,846,286,848
1_bn1                                                 [64]   [1024, 64, 112, 112]         128     3,288,334,336
2_relu                                                   -   [1024, 64, 112, 112]           0                 0
3_maxpool                                                -     [1024, 64, 56, 56]           0                 0
4_layer1.0.Conv2d_conv1                     [64, 64, 1, 1]     [1024, 64, 56, 56]       4,096    13,153,337,344
5_layer1.0.BatchNorm2d_bn1                            [64]     [1024, 64, 56, 56]         128       822,083,584
6_layer1.0.ReLU_relu                                     -     [1024, 64, 56, 56]           0                 0
7_layer1.0.Conv2d_conv2                     [64, 64, 3, 3]     [1024, 64, 56, 56]      36,864   118,380,

In [11]:
train_epoch_acc = []
train_epoch_losses = []
valid_epoch_losses = []
valid_epoch_acc = []

for epoch in range(config['epochs']):
    model.train()
    batch_loss = []
    batch_acc = []
    
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_loss.append(loss.item())
        _, preds = torch.max(outputs, 1)
        acc = (preds == labels).sum().item() / labels.size(0)
        batch_acc.append(acc)
        
    train_epoch_acc.append(np.mean(batch_acc))
    train_epoch_losses.append(np.mean(batch_loss))
    
    
    model.eval()
    batch_loss = []
    batch_acc = []
    
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            batch_loss.append(loss.item())
            _, preds = torch.max(outputs, 1)
            acc = (preds == labels).sum().item() / labels.size(0)
            batch_acc.append(acc)
            
    valid_epoch_losses.append(np.mean(batch_loss))
    valid_epoch_acc.append(np.mean(batch_acc))
    print(f'[Epoch {epoch:02d}] Train loss: {train_epoch_losses[-1]:.4f}, valid loss = {valid_epoch_losses[-1]:.4f} Train acc {train_epoch_acc[-1]:.4f} Valid acc {valid_epoch_acc[-1]:.4f}')
    
fig, ax = plt.subplots(1, 2, figsize=(20, 4))
ax[0].plot(train_epoch_losses)
ax[0].plot(valid_epoch_losses)
ax[0].set_ylim((0, np.max(train_epoch_losses.extend(valid_epoch_losses))))
ax[0].set_title('Valid loss history')

ax[1].plot(train_epoch_acc)
ax[1].plot(valid_epoch_acc)
ax[1].set_ylim(0, 1.1)
ax[1].set_title('Acc history')

[Epoch 00] Train loss: 1.3556, valid loss = 0.9476 Train acc 0.6450 Valid acc 0.8221


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model.pt')